<a href="https://colab.research.google.com/github/vrmusketeers/DeepLearningProject/blob/main/notebooks/Generate_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro
In this notebook we load SVGs for thousands of house floorplans which contain information about room placement and size. We convert those SVGs to images where certain rooms are colored a certain color depending on their room type, such as bedrooms, kitchens, bathrooms, etc. Certain built-in furniture items are also colored in, such as stoves, toilets, sinks, etc. We also generate a textual description of the house depending how many floors and rooms there are per floor.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vrmusketeers/DeepLearningProject/blob/master/notebooks/Generate_Data.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download CubiCasa dataset

In [ ]:
!wget https://zenodo.org/record/2613548/files/cubicasa5k.zip

--2021-04-16 14:52:24--  https://zenodo.org/record/2613548/files/cubicasa5k.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5469495706 (5.1G) [application/octet-stream]
Saving to: ‘cubicasa5k.zip’

cubicasa5k.zip      100%[===================>]   5.09G  3.79MB/s    in 23m 47s 

2021-04-16 15:16:13 (3.65 MB/s) - ‘cubicasa5k.zip’ saved [5469495706/5469495706]



In [ ]:
! cp /content/cubicasa5k.zip '/content/drive/MyDrive/CMPE 258 Deep Learning/Data Prep CubiCasa/cubicasa5k.zip'

In [ ]:
!unzip '/content/drive/MyDrive/CMPE 258 Deep Learning/Data Prep CubiCasa/cubicasa5k.zip' -d /content

Streaming output truncated to the last 5000 lines.
   creating: /content/cubicasa5k/high_quality_architectural/4014/
  inflating: /content/cubicasa5k/high_quality_architectural/4014/F1_scaled.png  
  inflating: /content/cubicasa5k/high_quality_architectural/4014/F1_original.png  
  inflating: /content/cubicasa5k/high_quality_architectural/4014/model.svg  
   creating: /content/cubicasa5k/high_quality_architectural/14296/
  inflating: /content/cubicasa5k/high_quality_architectural/14296/F1_scaled.png  
  inflating: /content/cubicasa5k/high_quality_architectural/14296/F1_original.png  
  inflating: /content/cubicasa5k/high_quality_architectural/14296/model.svg  
   creating: /content/cubicasa5k/high_quality_architectural/14295/
  inflating: /content/cubicasa5k/high_quality_architectural/14295/F2_scaled.png  
  inflating: /content/cubicasa5k/high_quality_architectural/14295/F1_scaled.png  
  inflating: /content/cubicasa5k/high_quality_architectural/14295/F1_original.png  
  inflating: /co

# Setup code to process SVGs into annotated images

## Use dataset util methods to load SVGs

In [ ]:
!git clone https://github.com/CubiCasa/CubiCasa5k.git

Cloning into 'CubiCasa5k'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 93 (delta 11), reused 0 (delta 0), pack-reused 74
Unpacking objects: 100% (93/93), done.


In [ ]:
!pip install svgpathtools num2words

     |████████████████████████████████| 71kB 3.9MB/s 
     |████████████████████████████████| 102kB 5.5MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 


In [ ]:
!mv /content/CubiCasa5k /content/CubiCasaSource
!touch /content/CubiCasaSource/__init__.py

In [ ]:
import sys
sys.path.append('/content/CubiCasaSource')

from skimage import transform
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import matplotlib.image as mpimg
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from floortrans.models import get_model
from floortrans.loaders import FloorplanSVG, DictToTensor, Compose, RotateNTurns, svg_utils
from floortrans.plotting import segmentation_plot, polygons_to_image, draw_junction_from_dict, discrete_cmap
from floortrans.post_prosessing import split_prediction, get_polygons, split_validation
from mpl_toolkits.axes_grid1 import AxesGrid
from num2words import num2words
from xml.dom import minidom

## Setup floorplan color schemes

In [ ]:
BACKGROUND_COLOR = '#f2f2f2'

legend = [
  {'region_name': 'Background', 'c': BACKGROUND_COLOR, 'shouldInclude': True},
  {'region_name': 'Yard', 'c': '#b3de69', 'shouldInclude': True},
  {'region_name': 'Wall', 'c': '#000000', 'shouldInclude': True},
  {'region_name': 'Kitchen', 'c': '#8dd3c7', 'shouldInclude': True},
  {'region_name': 'Living Room', 'c': '#fdb462', 'shouldInclude': True},
  {'region_name': 'Bedroom', 'c': '#fccde5', 'shouldInclude': True},
  {'region_name': 'Bathroom', 'c': '#80b1d3', 'shouldInclude': True},
  {'region_name': 'Hallway', 'c': '#808080', 'shouldInclude': True},
  {'region_name': 'Railing', 'c': '#663300', 'shouldInclude': False},
  {'region_name': 'Storage room', 'c': '#cc99ff', 'shouldInclude': True},
  {'region_name': 'Garage', 'c': '#577a4d', 'shouldInclude': True},
  {'region_name': 'Other room', 'c': '#ffffb3', 'shouldInclude': True},

  {'region_name': 'No Furnishing', 'c': '#351f13', 'shouldInclude': False},
  {'region_name': 'Window', 'c': '#d5f5fb', 'shouldInclude': True},
  {'region_name': 'Door', 'c': '#cc6600', 'shouldInclude': True},
  {'region_name': 'Closet', 'c': '#ff6666', 'shouldInclude': True},
  {'region_name': 'Electrical Appliance', 'c': '#999900', 'shouldInclude': True},
  {'region_name': 'Toilet', 'c': '#eeeeee', 'shouldInclude': True},
  {'region_name': 'Sink', 'c': '#eeeeee', 'shouldInclude': True},
  {'region_name': 'Sauna Bench', 'c': '#7588c4', 'shouldInclude': True},
  {'region_name': 'Fireplace', 'c': '#cc0000', 'shouldInclude': True},
  {'region_name': 'Bathtub', 'c': '#00d6a4', 'shouldInclude': True},
  {'region_name': 'Chimney', 'c': '#94a4aa', 'shouldInclude': True},
  {'region_name': 'Other furnishing', 'c': '#79847e', 'shouldInclude': False},
]

cpool = [l['c'] if l['shouldInclude'] else BACKGROUND_COLOR for l in legend]
cmap3 = colors.ListedColormap(cpool, 'rooms_test')
cm.register_cmap(cmap=cmap3)

NUM_ROOM_TYPES = 12

## Create mapping for room ID to room name

In [ ]:
rooms_selected = {"Alcove": 11,
                  "Attic": 11,
                  "Ballroom": 11,
                  "Bar": 11,
                  "Basement": 11,
                  "Bath": 6,
                  "Bedroom": 5,
                  "CarPort": 10,
                  "Church": 11,
                  "Closet": 9,
                  "ConferenceRoom": 11,
                  "Conservatory": 11,
                  "Counter": 11,
                  "Den": 11,
                  "Dining": 4,
                  "DraughtLobby": 7,
                  "DressingRoom": 9,
                  "EatingArea": 4,
                  "Elevated": 11,
                  "Elevator": 11,
                  "Entry": 7,
                  "ExerciseRoom": 11,
                  "Garage": 10,
                  "Garbage": 11,
                  "Hall": 11,
                  "HallWay": 7,
                  "HotTub": 11,
                  "Kitchen": 3,
                  "Library": 11,
                  "LivingRoom": 4,
                  "Loft": 11,
                  "Lounge": 4,
                  "MediaRoom": 11,
                  "MeetingRoom": 11,
                  "Museum": 11,
                  "Nook": 11,
                  "Office": 11,
                  "OpenToBelow": 11,
                  "Outdoor": 1,
                  "Pantry": 11,
                  "Reception": 11,
                  "RecreationRoom": 11,
                  "RetailSpace": 11,
                  "Room": 11,
                  "Sanctuary": 11,
                  "Sauna": 6,
                  "ServiceRoom": 11,
                  "ServingArea": 11,
                  "Skylights": 11,
                  "Stable": 11,
                  "Stage": 11,
                  "StairWell": 11,
                  "Storage": 9,
                  "SunRoom": 11,
                  "SwimmingPool": 11,
                  "TechnicalRoom": 11,
                  "Theatre": 11,
                  "Undefined": 11,
                  "UserDefined": 11,
                  "Utility": 11,
                  "Background": 0,  # Not in data. The default outside label
                  "Wall": 2,
                  "Railing": 8}
icons_selected = {"Window": 1,
                  "Door": 2,
                  "Closet": 3,
                  "ClosetRound": 3,
                  "ClosetTriangle": 3,
                  "CoatCloset": 3,
                  "CoatRack": 3,
                  "CounterTop": 3,
                  "Housing": 3,
                  "ElectricalAppliance": 4,
                  "WoodStove": 4,
                  "GasStove": 4,
                  "Toilet": 5,
                  "Urinal": 5,
                  "SideSink": 6,
                  "Sink": 6,
                  "RoundSink": 6,
                  "CornerSink": 6,
                  "DoubleSink": 6,
                  "DoubleSinkRight": 6,
                  "WaterTap": 6,
                  "SaunaBenchHigh": 7,
                  "SaunaBenchLow": 7,
                  "SaunaBenchMid": 7,
                  "SaunaBench": 7,
                  "Fireplace": 8,
                  "FireplaceCorner": 8,
                  "FireplaceRound": 8,
                  "PlaceForFireplace": 8,
                  "PlaceForFireplaceCorner": 8,
                  "PlaceForFireplaceRound": 8,
                  "Bathtub": 9,
                  "BathtubRound": 9,
                  "Chimney": 10,
                  "Misc": 0,
                  "BaseCabinetRound": 0,
                  "BaseCabinetTriangle": 0,
                  "BaseCabinet": 0,
                  "WallCabinet": 0,
                  "Shower": 0,
                  "ShowerCab": 0,
                  "ShowerPlatform": 0,
                  "ShowerScreen": 0,
                  "ShowerScreenRoundRight": 0,
                  "ShowerScreenRoundLeft": 0,
                  "Jacuzzi": 0
                  }

rooms = {}
for key, val in rooms_selected.items():
  rooms[val] = rooms.get(val, []) + [key]

furniture = {}
for key, val in icons_selected.items():
  furniture[val + NUM_ROOM_TYPES] = furniture.get(val + NUM_ROOM_TYPES, []) + [key]

legend = {**rooms, **furniture}
legend

{0: ['Background'],
 1: ['Outdoor'],
 2: ['Wall'],
 3: ['Kitchen'],
 4: ['Dining', 'EatingArea', 'LivingRoom', 'Lounge'],
 5: ['Bedroom'],
 6: ['Bath', 'Sauna'],
 7: ['DraughtLobby', 'Entry', 'HallWay'],
 8: ['Railing'],
 9: ['Closet', 'DressingRoom', 'Storage'],
 10: ['CarPort', 'Garage'],
 11: ['Alcove',
  'Attic',
  'Ballroom',
  'Bar',
  'Basement',
  'Church',
  'ConferenceRoom',
  'Conservatory',
  'Counter',
  'Den',
  'Elevated',
  'Elevator',
  'ExerciseRoom',
  'Garbage',
  'Hall',
  'HotTub',
  'Library',
  'Loft',
  'MediaRoom',
  'MeetingRoom',
  'Museum',
  'Nook',
  'Office',
  'OpenToBelow',
  'Pantry',
  'Reception',
  'RecreationRoom',
  'RetailSpace',
  'Room',
  'Sanctuary',
  'ServiceRoom',
  'ServingArea',
  'Skylights',
  'Stable',
  'Stage',
  'StairWell',
  'SunRoom',
  'SwimmingPool',
  'TechnicalRoom',
  'Theatre',
  'Undefined',
  'UserDefined',
  'Utility'],
 12: ['Misc',
  'BaseCabinetRound',
  'BaseCabinetTriangle',
  'BaseCabinet',
  'WallCabinet',
  'Sh

## Create util methods to parse SVG and color rooms
* Parse floorplan SVG for different floors and room regions
* Create text description for each house's floorplan based on the number floors and room regions present
* Integrates above `get_regions` and `generate_floorplan_description` given folder to generate data from floorplan SVGs
* Save generated image processed from SVG

# Generate images for dataset

In [ ]:
# Parse floorplan SVG for different floors and room regions
def get_regions(root_dir, instance_dir):
  path = root_dir + '/' + instance_dir + '/model.svg'
  svg = minidom.parse(path)
  house = {}
  
  for component in svg.getElementsByTagName('g'):
    floor_elem = component.getAttribute("class")
    if "Floorplan " in component.getAttribute("class"):
      floor_num = int(component.getAttribute("class").split(" ")[1].split('-')[1])
      house[floor_num] = house.get(floor_num, {})

      for e in component.getElementsByTagName('g'):
        if "Space " in e.getAttribute("class"):
          name_list = e.getAttribute("class").split(" ")
          room_type = name_list[1]
          index = svg_utils.get_room_number(e, rooms_selected)
          region_name = legend[index]['region_name']
          house[floor_num][region_name] = house[floor_num].get(region_name, 0) + 1
  return house

# Create text description for each house's floorplan based on the number floors 
# and room regions present
def generate_floorplan_description(house):
  num_floors = len(house.keys())
  s = 'The house has {} floor'.format(num2words(num_floors))
  if num_floors > 1:
    s += 's'
  s += '.'

  for floor in range(1, num_floors + 1):
    s += ' The {} floor has'.format(num2words(floor, to='ordinal'))
    house_floor = house[floor]
    for l in legend:
      room_name = l['region_name']
      if room_name in house_floor and l['shouldInclude']:
        s += ' {} {}'.format(num2words(house_floor[room_name]), room_name.lower())
        if house_floor[room_name] > 1:
          s += 's'

    s += '.'

  return s

# Integrates above get_regions and generate_floorplan_description given folder
# to generate data from floorplan SVGs
def get_text(root_dir, instance_folder):
  house = get_regions(root_dir, instance_folder)
  return len(house.keys()), generate_floorplan_description(house)

# Save generated image processed from SVG
def gen_annotated_img(label, save_path):
  label_np = label.data.numpy()[0]
  new_label_np = np.where(label_np[1] != 0,  label_np[1] + NUM_ROOM_TYPES, label_np[0])

  plt.figure(figsize=(10,10))
  plt.axis('off')
  rseg = plt.imshow(new_label_np, cmap='rooms_test', vmin=0, vmax=len(legend)-0.1)
  plt.savefig(save_path)
  plt.clf()

## Load data

In [ ]:
data_folder = '/content/cubicasa5k/'
data_file = 'train.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)

In [ ]:
val = next(data_iter)
junctions = val['heatmaps']
folder = val['folder'][0]
image = val['image']
label = val['label']

file_id = folder.split('/')[2]
instance_dir = '/high_quality_architectural/{}'.format(file_id)
print(instance_dir)

/high_quality_architectural/6165


## Render example of generated training data

In [ ]:
# Show orig SVG image

np_img = np.moveaxis(image[0].cpu().data.numpy(), 0, -1) / 2 + 0.5
plt.figure(figsize=(10,10))
plt.title('Source Image', fontsize=20)
plt.axis('off')
plt.imshow(np_img)
plt.show()

# Show generated annotated image to use for training

label_np = label.data.numpy()[0]
new_label_np = np.where(label_np[1] != 0,  label_np[1] + NUM_ROOM_TYPES, label_np[0])

plt.figure(figsize=(10,10))
plt.axis('off')
rseg = plt.imshow(new_label_np, cmap='rooms_test', vmin=0, vmax=len(legend)-0.1)
cbar = plt.colorbar(rseg, ticks=(np.arange(len(legend) + 1)+ 0.5 ), fraction=0.1, pad=0.05)
cbar.ax.set_yticklabels([l['region_name'] for l in legend], fontsize=15)
plt.show()

text = get_text(data_folder, instance_dir)
print(text)

(1, 'The house has one floor. The first floor has one yard one kitchen two bedrooms two other rooms.')


## Show SVG as example

In [ ]:
!cat /content/cubicasa5k/{instance_dir}/model.svg

<?xml version="1.0"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" height="1369.6700439453125" version="1.1" width="959.7100524902344" id="temp" viewBox="0 0 959.7100524902344 1369.6700439453125"><desc>Powered by CubiCasa</desc><defs/><g id="Model" class="Model v1-1"><g class="Floor"><g id="Floor-1" class="Floorplan Floor-1"><g id="54b17949-5218-468d-bed7-3d2a913741dc" fill="#ffffff" stroke="#ffffff" style="fill-opacity: 1; stroke-opacity: 1; stroke-width: 0.2;" class="Space Bedroom"><polygon points="432.65,744.65 432.72,744.65 432.72,824.16 432.65,824.16 431.72,881.94 216.58,881.94 216.58,673.75 432.65,673.75 "/><g transform="matrix(1,0,0,1,216.5842,673.7539)" class="Dimension"><g transform="matrix(1,0,0,1,0,0)" style="display: none;" class="Visual"><g id="mark1" fill="#000000" stroke="none" transform="matrix(1,0,0,1,108.0662,0)" style="fill-opacity: 1; pointer-events: visible;" class="DimensionMark"><polygon points="-5,5,0,0,5,5"/></g><g id="mark

## Process all SVGs into annotated images with room regions and house text description

In [ ]:
!rm -rf /content/annotated_cubicasa
!mkdir -p /content/annotated_cubicasa

In [ ]:
data_iter = iter(data_loader)
num_iter = len(data_iter)
print(num_iter)

i = 0
for i in range(num_iter):
  val = next(data_iter)

  if not val:
    print('{}: Error loading'.format(i))
    i += 1
    continue

  folder = val['folder'][0]
  file_id = folder.split('/')[2]
  label = val['label']

  num_floors, text = get_text(data_folder, folder)

  instance_dir = '/content/annotated_cubicasa/{}-Floors/{}'.format(num_floors, file_id)

  if not os.path.isdir(instance_dir):
    pathlib.Path(instance_dir).mkdir(parents=True, exist_ok=True) 
  else:
    print('{}: Skipping {}'.format(i, instance_dir))
    i += 1
    continue

  with open('{}/{}.txt'.format(instance_dir, file_id), "w") as file:
    file.write(text)

  gen_annotated_img(label, '{}/{}.jpg'.format(instance_dir, file_id))

  print('{}: Annotated {}'.format(i, file_id))
  i += 1

4200
0: Skipping /content/annotated_cubicasa/1-Floors/6044
1: Skipping /content/annotated_cubicasa/3-Floors/2564
2: Skipping /content/annotated_cubicasa/1-Floors/6165
3: Skipping /content/annotated_cubicasa/2-Floors/1021
4: Skipping /content/annotated_cubicasa/2-Floors/5507
5: Skipping /content/annotated_cubicasa/2-Floors/1269
6: Skipping /content/annotated_cubicasa/2-Floors/5988
7: Skipping /content/annotated_cubicasa/1-Floors/2123
8: Skipping /content/annotated_cubicasa/1-Floors/6293
9: Skipping /content/annotated_cubicasa/1-Floors/925
10: Skipping /content/annotated_cubicasa/1-Floors/1171
11: Skipping /content/annotated_cubicasa/1-Floors/9333
12: Skipping /content/annotated_cubicasa/2-Floors/8815
13: Skipping /content/annotated_cubicasa/2-Floors/1784
14: Skipping /content/annotated_cubicasa/1-Floors/6335
15: Skipping /content/annotated_cubicasa/2-Floors/4947
16: Skipping /content/annotated_cubicasa/2-Floors/4501
17: Skipping /content/annotated_cubicasa/1-Floors/3796
18: Skipping /co

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


132: Annotated 1169
133: Annotated 748
134: Annotated 534
135: Annotated 8139
136: Annotated 1491
137: Annotated 3604
138: Annotated 9945
139: Annotated 6030
140: Annotated 1700
141: Annotated 8752
142: Annotated 78
143: Annotated 9027
144: Annotated 7802
145: Annotated 8809
146: Annotated 3883
147: Annotated 9557
148: Annotated 8741
149: Annotated 8100
150: Annotated 4790
151: Annotated 9633
152: Annotated 1207
153: Annotated 1579
154: Annotated 7532
155: Annotated 4906
156: Annotated 1457
157: Annotated 2055
158: Annotated 5887
159: Annotated 2144
160: Annotated 1763
161: Annotated 3118
162: Annotated 9262
163: Annotated 1725
164: Annotated 7805
165: Annotated 4064
166: Annotated 1426
167: Annotated 5915
168: Annotated 5061
169: Annotated 6062
170: Annotated 8803
171: Annotated 6334
172: Annotated 6300
173: Annotated 4821
174: Annotated 8569
175: Annotated 5793
176: Annotated 6368
177: Annotated 3649
178: Annotated 3204
179: Annotated 1014
180: Annotated 82
181: Annotated 253
182: An

/content/CubiCasaSource/floortrans/loaders/house.py:952: RuntimeWarning: divide by zero encountered in double_scalars
  res = res / float(i)


1001: Annotated 4520
1002: Annotated 6376
1003: Annotated 9034
1004: Annotated 3085
1005: Annotated 8332
1006: Annotated 4884
1007: Annotated 9677
1008: Annotated 8252
1009: Annotated 577
1010: Annotated 3849
1011: Annotated 4584
1012: Annotated 3184
1013: Annotated 3505
1014: Annotated 905
1015: Annotated 1266
1016: Annotated 9019
1017: Annotated 1575
1018: Annotated 7105
1019: Annotated 4971
1020: Annotated 7718
1021: Annotated 5693
1022: Annotated 5805
1023: Annotated 9881
1024: Annotated 2113
1025: Annotated 285
1026: Annotated 716
1027: Annotated 9098
1028: Annotated 1099
1029: Annotated 6616
1030: Annotated 1589
1031: Annotated 8660
1032: Annotated 9735
1033: Annotated 9050
1034: Annotated 678
1035: Annotated 6718
1036: Annotated 7034
1037: Annotated 6261
1038: Annotated 3717
1039: Annotated 4722
1040: Annotated 8199
1041: Annotated 1974
1042: Annotated 6523
1043: Annotated 1525
1044: Annotated 6645
1045: Annotated 302
1046: Annotated 3957
1047: Annotated 504
1048: Annotated 5884

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

In [ ]:
!ls /content/annotated_cubicasa/* | wc -l

4207


# Zip and store results

In [ ]:
!rm /content/annotated_cubicasa.zip
!zip -r /content/annotated_cubicasa.zip /content/annotated_cubicasa

Streaming output truncated to the last 5000 lines.
  adding: content/annotated_cubicasa/1-Floors/1004/1004.jpg (deflated 37%)
  adding: content/annotated_cubicasa/1-Floors/1004/1004.txt (deflated 35%)
  adding: content/annotated_cubicasa/1-Floors/11088/ (stored 0%)
  adding: content/annotated_cubicasa/1-Floors/11088/11088.txt (deflated 34%)
  adding: content/annotated_cubicasa/1-Floors/11088/11088.jpg (deflated 38%)
  adding: content/annotated_cubicasa/1-Floors/14445/ (stored 0%)
  adding: content/annotated_cubicasa/1-Floors/14445/14445.jpg (deflated 41%)
  adding: content/annotated_cubicasa/1-Floors/14445/14445.txt (deflated 37%)
  adding: content/annotated_cubicasa/1-Floors/8683/ (stored 0%)
  adding: content/annotated_cubicasa/1-Floors/8683/8683.jpg (deflated 48%)
  adding: content/annotated_cubicasa/1-Floors/8683/8683.txt (deflated 37%)
  adding: content/annotated_cubicasa/1-Floors/107/ (stored 0%)
  adding: content/annotated_cubicasa/1-Floors/107/107.jpg (deflated 39%)
  adding: c

In [ ]:
!unzip -l /content/annotated_cubicasa.zip | less 

Archive:  /content/annotated_cubicasa.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
        0  2021-04-16 18:19   content/annotated_cubicasa/
        0  2021-04-16 19:45   content/annotated_cubicasa/3-Floors/
        0  2021-04-16 19:10   content/annotated_cubicasa/3-Floors/12404/
    55557  2021-04-16 19:10   content/annotated_cubicasa/3-Floors/12404/12404.jp g
      304  2021-04-16 19:10   content/annotated_cubicasa/3-Floors/12404/12404.tx t
        0  2021-04-16 19:42   content/annotated_cubicasa/3-Floors/11436/
    41906  2021-04-16 19:42   content/annotated_cubicasa/3-Floors/11436/11436.jp g
      315  2021-04-16 19:42   content/annotated_cubicasa/3-Floors/11436/11436.tx t
        0  2021-04-16 18:46   content/annotated_cubicasa/3-Floors/4963/
      168  2021-04-16 18:46   content/annotated_cubicasa/3-Floors/4963/4963.txt
    44010  2021-04-16 18:46   content/annotated_cubicasa/3-Floors/4963/4963.jpg
        0  2021-04-16 19:29   content/annotated_c

In [ ]:
! cp /content/annotated_cubicasa.zip '/content/drive/MyDrive/CMPE 258 Deep Learning/CubiCasa/annotated_cubicasa.zip'